# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load csv
weather_data = pd.read_csv("../WeatherPy/Output_Data/cities.csv")
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Nizhnevartovsk,60.9344,76.5531,30.20,69,0,4.47,RU,1620077501
1,Khatanga,71.9667,102.5000,23.36,82,88,13.31,RU,1620077256
2,Hithadhoo,-0.6000,73.0833,83.35,70,100,5.35,MV,1620077501
3,Hobart,-42.8794,147.3294,41.00,93,20,9.22,AU,1620077502
4,Jamestown,42.0970,-79.2353,62.01,82,90,14.97,US,1620077502
...,...,...,...,...,...,...,...,...,...
577,Wilmington,34.2257,-77.9447,84.20,61,1,11.50,US,1620077589
578,Tyukalinsk,55.8732,72.1967,39.94,53,4,6.15,RU,1620077641
579,Caraz,-9.0486,-77.8100,61.50,92,98,5.17,PE,1620077641
580,Esfarāyen,37.0765,57.5101,55.40,88,90,2.30,IR,1620077641


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)

# Use Lat and Lng as locations
locations = weather_data[["Lat", "Lng"]]

# Use Humidity as weight
weights = weather_data["Humidity"]

In [4]:
# Pass API key
gmaps.configure(api_key = g_key)

# Create plot
fig = gmaps.figure(center=[0, 0], zoom_level=2)

# Create and add heat layer
heat_layer = gmaps.heatmap_layer(locations, weights, dissipating = False, max_intensity = 100, point_radius = 3)
fig.add_layer(heat_layer)

# Display plot
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Identify cities that meet criteria: wind speed < 5 mph, cloudiness < 10, max temp between 75 and 90
criteria_df = weather_data.loc[(weather_data["Wind Speed"]<=5) & (weather_data["Cloudiness"]<=10) & (weather_data["Max Temp"]>=75) & (weather_data["Max Temp"]<=90)].dropna()
criteria_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
71,Faanui,-16.4833,-151.7500,81.03,69,5,2.21,PF,1620077364
124,São Miguel do Araguaia,-13.2750,-50.1628,76.91,56,5,3.83,BR,1620077529
170,Haikou,20.0458,110.3417,77.00,94,0,4.47,CN,1620077540
186,Saint George,37.1041,-113.5841,75.99,18,1,4.61,US,1620077306
368,Esna,25.2934,32.5540,87.80,22,0,3.44,EG,1620077590
431,Jizan,17.3333,42.6667,84.20,74,0,3.44,SA,1620077606
447,Santa Maria,-29.6842,-53.8069,86.00,45,0,3.44,BR,1620077610
488,Vila Velha,-20.3297,-40.2925,77.00,83,0,4.61,BR,1620077619


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create hotel DataFrame
hotel_df = criteria_df.loc[:, ["City", "Lat", "Lng", "Country"]]

# Add Hotel Name column
hotel_df["Hotel Name"]= ""

# Display DataFrame
hotel_df

,City,Lat,Lng,Country,Hotel Name
71,Faanui,-16.4833,-151.7500,PF,
124,São Miguel do Araguaia,-13.2750,-50.1628,BR,
170,Haikou,20.0458,110.3417,CN,
186,Saint George,37.1041,-113.5841,US,
368,Esna,25.2934,32.5540,EG,
431,Jizan,17.3333,42.6667,SA,
447,Santa Maria,-29.6842,-53.8069,BR,
488,Vila Velha,-20.3297,-40.2925,BR,


In [9]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type": "hotel", "keyword": "hotel", "radius": 5000, "key": g_key}

for index, row in hotel_df.iterrows():
     # Get latitude, longitude, and name of city from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    
    # Add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # Assemble URL and make API request
    print(f"Getting Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    # Get results
    results = response["results"]
    
    # Save hotel name to DataFrame
    try:
        print(f"Closest hotel in {city} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # If no hotel, show that it's missing
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
# Print end of search once searching is completed
print("-------End of Search-------")

Getting Results for Index 71: Faanui.
Missing field/result... skipping.
------------
Getting Results for Index 124: São Miguel do Araguaia.
Missing field/result... skipping.
------------
Getting Results for Index 170: Haikou.
Missing field/result... skipping.
------------
Getting Results for Index 186: Saint George.
Missing field/result... skipping.
------------
Getting Results for Index 368: Esna.
Missing field/result... skipping.
------------
Getting Results for Index 431: Jizan.
Missing field/result... skipping.
------------
Getting Results for Index 447: Santa Maria.
Missing field/result... skipping.
------------
Getting Results for Index 488: Vila Velha.
Missing field/result... skipping.
------------
-------End of Search-------


In [10]:
# Display DataFrame with hotel names
hotel_df

,City,Lat,Lng,Country,Hotel Name
71,Faanui,-16.4833,-151.7500,PF,
124,São Miguel do Araguaia,-13.2750,-50.1628,BR,
170,Haikou,20.0458,110.3417,CN,
186,Saint George,37.1041,-113.5841,US,
368,Esna,25.2934,32.5540,EG,
431,Jizan,17.3333,42.6667,SA,
447,Santa Maria,-29.6842,-53.8069,BR,
488,Vila Velha,-20.3297,-40.2925,BR,


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))